In [1]:
#Import the necessary packages (some might be unecessary, but shh)

from astropy.table import Table
from matplotlib import pyplot as plt
%matplotlib inline
import os
import numpy as np
from astropy.io.ascii import masked
from astropy.io import ascii
import glob
from astropy.io import fits
import wget
import matplotlib.image as mpimg
from astropy.wcs import WCS
from scipy.stats import scoreatpercentile
from astropy.visualization import simple_norm
from reproject import reproject_interp
import sys
from IPython.display import clear_output
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
from photutils.aperture import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization import ImageNormalize
from astropy.visualization import LogStretch
from astropy.wcs import WCS
import glob
import os
from scipy.stats import scoreatpercentile
import astropy.units as u
from IPython.display import clear_output
from matplotlib import colors
import warnings
import csv
warnings.filterwarnings('ignore')

mycolors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
#Set the home path
os.environ['HOME'] ='C:/Users/USER/Documents/GitHub' #general path for all the coding space
homedir = os.getenv("HOME")
tabledir = homedir+'/Herschel summary/tables' #tables from the VFS
datadir = homedir+'/Herschel summary/html resource' #outputting the pngs

In [ ]:
#get optical cutouts pngs from the Legacy Survey
csv_file = tabledir+'/Photometrytesting.csv'
galaxy = Table.read(csv_file)
pixscale=1   #standard
pscale = 0
xsize = 0
for i in range(len(galaxy)):
    galaxy_name = str(galaxy['GALAXY'][i])
    path =  datadir+'/pipeline/'+galaxy_name
    destination_folder = path+'/HPPUNIMAPR/'
    partial_name = 'hpacs_25HPPUNIMAPR'
    found_files = find_files(destination_folder, partial_name)
    VFID = f"VFID{int(galaxy['VF_ID'][i]):04d}"
    filename_LS = datadir+'/png/'+VFID+'-'+galaxy_name+'-LS.jpg'
    RA = galaxy['RA_MOMENT'][i]
    DEC = galaxy['DEC_MOMENT'][i]
    if os.path.exists(path):
        if found_files:
            found_file = found_files[0]
            image, head = fits.getdata(found_file, header=True)
            wcs_info = WCS(head) 
            pscale=np.abs(float(head['CDELT1']))   #grab transformation matrix of Herschel image
            xsize=np.abs(int(head['NAXIS1']))   #grab length of Herschel image
            xsize_arcsec=pscale*3600*xsize   #length convert to arcseconds
            imsize=int(xsize_arcsec/pixscale)   #convert length to an integer
            imsize=str(imsize)   #convert integer length to a...string       
            image_url = f'https://www.legacysurvey.org/viewer/cutout.jpg?ra={RA}&dec={DEC}&layer=ls-dr9&size={imsize}&pixscale={1}'
            if os.path.exists(filename_LS):
                os.remove(filename_LS)
                imageLS = wget.download(image_url,out=filename_LS)
            else:
                imageLS = wget.download(image_url,out=filename_LS)
    else:
        print(f"Galaxy not found: {VFID}-{galaxy_name}")
print('done')

In [ ]:
#Writes the output html file in 
# Open the HTML file in write mode
with open("output.html", "w") as html:
    # Write the HTML code line by line
    html.write('<html><body>\n')
    html.write('<title>Herschel images</title>\n')
    html.write('<style type="text/css">\n')
    html.write('table, td, th {padding: 5px; text-align: center; border: 2px solid black;}\n')
    html.write('p {display: inline-block;;}\n')
    html.write('</style>\n')
    html.write('<table><tr><th>#</th><th>VFID</th><th>Name</th><th>Legacy image</th><th>Herschel-UnimapBlue (70microns)</th><th>Herschel-UnimapBlue (100microns)</th><th>Herschel-UnimapRed (160microns)</th><th>RA</th><th>DEC</th><th>B/G significant mask</th><th>R significant mask</th></tr></p>')
    
    # Read the CSV file containing the galaxy data
    galaxy = Table.read(tabledir+'/Photometrytesting2.csv')

    for i in range(len(galaxy)):
        n = str(i+1)
        galaxy_name = str(galaxy['GALAXY'][i])
        path = datadir + '\\pipeline\\' + galaxy_name
        VFID = f"VFID{int(galaxy['VF_ID'][i]):04d}"
        RA = f"{galaxy['RA_MOMENT'][i]:.3f}"
        DEC = f"{galaxy['DEC_MOMENT'][i]:.3f}"
        
        # Fetch the percentage values from the table
        good_percentage_blue_green = galaxy['good_perc_blue_green'][i]
        good_percentage_red = galaxy['good_perc_red'][i]
        
        # Check if values are less than 0.95 and set the mask accordingly
        bg_significant_mask = 'True' if good_percentage_blue_green < 0.95 else 'False'
        r_significant_mask = 'True' if good_percentage_red < 0.95 else 'False'
        
        if os.path.exists(path):
            html.write('<tr><td>'+n+'</td><td><a href="galaxy/html/'+VFID+'-'+galaxy_name+'.html">'+VFID+'</td><td>'+galaxy_name+'</td><td><a href="galaxy/png/'+VFID+'-'+galaxy_name+'-LS.jpg"><img src="galaxy/png/'+VFID+'-'+galaxy_name+'-LS.jpg" alt="'+VFID+'-'+galaxy_name+'-LS.jpg" height="auto" width="100%"></a></td><td><a href="galaxy/png/'+VFID+'-'+galaxy_name+'blue.png"><img src="galaxy/png/'+VFID+'-'+galaxy_name+'blue.png" alt="'+VFID+'-'+galaxy_name+'blue.jpg" height="auto" width="100%"></a></td><td><a href="galaxy/png/'+VFID+'-'+galaxy_name+'green.png"><img src="galaxy/png/'+VFID+'-'+galaxy_name+'green.png" alt="'+VFID+'-'+galaxy_name+'green.jpg" height="auto" width="100%"></a></td><td><a href="galaxy/png/'+VFID+'-'+galaxy_name+'red.png"><img src="galaxy/png/'+VFID+'-'+galaxy_name+'red.png" alt="'+VFID+'-'+galaxy_name+'red.jpg" height="auto" width="100%"></a></td><td>'+RA+'</td><td>'+DEC+'</td><td>'+bg_significant_mask+'</td><td>'+r_significant_mask+'</td></tr>\n')
        else:
            continue

    html.write('</tr></table>\n')
    html.write('<br /><br />\n')
    html.write('</html></body>\n')

print("HTML code has been written to output.html")

Refer to the photoemtry notebook for generation of the individual HTML files, as I want the images from there first.